# (Task 1) Importing and filling the data
We're going to be abusing `pandas`, a pretty [standard tool for managing datasets for analysis](https://pandas.pydata.org/docs/). For the importing of the data we'll just read csv files into [dataframes](https://pandas.pydata.org/docs/user_guide/dsintro.html#basics-dataframe) directly and handle missing values [leaning on `pandas`](https://pandas.pydata.org/docs/user_guide/missing_data.html) by specifying the "NA values", and filling them in via respective column means. 

This assignment gives us the impression that data is presented nicely to us. But in reality... that data cleaning is a very sizeable task in of itself.

In [139]:
import pandas as pd
from sklearn import tree

# WARN: critical assumption here is missing data is denoted via '?', this was provided via Piazza clarification at
# at the time of the assignment, but we would absolutely need to do some data cleaning.
df = pd.read_csv('datasets/website-phishing.csv', na_values='?') 
df = df.fillna(df.mean())

# Training/Testing Sets
We will split here the training and testing datasets. Going old school with the 70/30 split. Not much to commentate here other than pointing out that it's important to;

- Shuffle the dataset, to remove any potential sorting that exists (mitigating training bias) and,
- Adhering to the golden rule of not polluting the training efforts with our testing dataset.

Ofcourse `scikit-learn` has a utility (aptly named [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)) here. As it very well should be, it's literally one of the most common procedures in these excercises to perform).

We finish here by conveniently seperating the features and labels too for easy access too (`tr_` prefixed to represent training).

In [122]:
from sklearn.model_selection import train_test_split

print("Overall size of the full dataset: ", df.size)
[training, testing] = train_test_split(df, shuffle=True, train_size=0.7, test_size=0.3)
print(f"Size of training set: {training.size} (%{training.size/df.size * 100}%)")
print(f"Size of testing set: {testing.size} (%{testing.size/df.size * 100}%)")

tr_features, tr_labels = training.iloc[:, :-1], training.iloc[:, -1]

Overall size of the full dataset:  342705
Size of training set: 239878 (%69.99547715965626%)
Size of testing set: 102827 (%30.004522840343732%)


# (Task 2a & 2b) Decision Stump & Unpruned Tree
And so we begin! I seperated these two subtasks out because they are adequately bland. But for future reference; for decision tree refresher I'd read the `scikit-learn` [chapter on decision trees](https://scikit-learn.org/stable/modules/tree.html). As we're about to dive right into the realms of our friend the [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

To pay respects to the lecturer Jörg Simon Wicker hosted in Summer-Autumn of 2023, we will fix the node splitting strategy for the length of this notebook to `criterion="entropy"`. That is quantifying data "impurity" via a flavour of the very well explained "entropy" function (Shannon entropy is used in `sci-kit learn`).

The [generalised definition](https://scikit-learn.org/stable/modules/tree.html#classification-criteria) is used here as we have datasets that have more than two possible labels (important distinction to make since we looked at only the binary classification definitions of entropy in the course).

In conjunction to fixing the `criterion` we will also for the length of this document fix `random_state=0` and `splitter="best"` as the implementation of the `DecisionTreeClassifier` within `scikit-learn` introduces randomness within selection of candidate features by default otherwise. We want deterministic behaviour to align with the teachings of this course. Hence all `DecisionTreeClassifier` instances will be set via;
```
DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best" ...)
```

The decision stump (called `stump_classifier` below) is just a decision tree classifier model with depth one. Now we're _technically_ assembling this tree via a "pre-pruning" technique of setting a `max_depth` of the tree, although sufficient for demonstrational purposes. The completely unpruned decision tree (called `unpruned_classifier` below) is tree that has no `max_depth` set (and no other hyperparameters set that would otherwise restrict the growth of this tree). Hence we have the following;

_NOTE: we log out the depth of the underlying constructed tree of the classifiers (perhaps unconventionally) via the property `.tree_.max_depth`_

In [140]:
stump_classifier = tree.DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best", max_depth=1)
stump_classifier.fit(tr_features, tr_labels)
print("The depth of tree assembled for stump_classifier:", stump_classifier.tree_.max_depth)

unpruned_classifier = tree.DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best")
unpruned_classifier.fit(tr_features, tr_labels)
print("The depth of the tree assembled for the unpruned_classifier:", unpruned_classifier.tree_.max_depth)

The depth of tree assembled for stump_classifier: 1
The depth of the tree assembled for the unpruned_classifier: 25


# (Task 2c) A Post Pruned Tree
We enter "pruning" the general technique of trimming the decision tree down in order to tackle the problem of "overfitting" - concretely in this section we'll take this `unpruned_tree` and apply the "cost complexity pruning" technique onto it.

Resources about this post pruning technique (as it's a novel technique relative to what has been covered in lectures);
- [Short video by StatQuest contextualised via Regression Tree's](https://www.youtube.com/watch?v=D0efHEJsfHo&t)
- [Chapter from `scikit-learn` docs directly](https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html)

It is a parameterised technique (takes in a "cost complexity" parameter, $\alpha$), hence open as a tunable hyper parameter. But the idea is that the general magnitude of $\alpha$ impacts how **aggressively** we prune this tree (_how many nodes we trim back_).

Technically, a post pruned tree here for sake of demonstration can simply be a product of this tree and a "strong enough" alpha to prune at least one decision node. Arbitrarily choosing this value is slightly innacurate as we need to garuantee some sort of actual "pruning" right? So we utulise this handy method on the instance of a `DecisionTreeClassifer` called `cost_complexity_pruning_path` that lists out possible alpha variables that _make an impact_. And from this selection we can arbitrarily choose one (for sake of demonstration) and apply the prune.

In [124]:
import random

classifier = tree.DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best")
path = classifier.cost_complexity_pruning_path(tr_features, tr_labels)

# NOTE: this purely a demonstration, we'd not just randomly choose the cost complexity pruning alpha like this
chosen_ccp_alpha = random.choice(path.ccp_alphas) 

post_pruned_classifier = tree.DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best", ccp_alpha=chosen_ccp_alpha)
post_pruned_classifier.fit(tr_features, tr_labels)
print("Total number of trimmed nodes: ", unpruned_classifier.tree_.node_count - post_pruned_classifier.tree_.node_count)

Total number of trimmed nodes:  416


# (Task 3) Hyperparameter (in our case just $\alpha$ ) search
Now the real cool shit (and the data science really begins). We want expand on what we found in our post pruned tree above... that is finding the $\alpha$ that _performs the best_ because above we surfaced a series of candidates (provided by the above `cost_complexity_pruning_path`), but just picked a random one!

So how can we find the best $\alpha$? Well we can split the training data set (that is still **strictly separated** from our `testing` dataset way at the beginning of this notebook) into the 70:30 ratio again, building two new sets, call them the `training` and `validation` sets. We'd do this because that allows us to iterate through each candidate $\alpha$ here, train it on the `training` set and test the result of this $\alpha$ tuned model on the `validation` set.

But as pointed out in the lecture notes, this restricts our training dataset quite a lot and impacts the ability for our models to generalise well (as our training dataset becomes less and less representative of our overall global dataset the smaller it gets). This caveat here is really only visible when contrasted against the [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) technique.

We covered this technique thoroughly in course lectures, effectively, we end up evaluating the regulated learning process under supervision of an $\alpha$ while training the model on the entire `training` set without this second layer of splitting using _folding_. In `scikit-learn`, we have the [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) utility; where given a list of hyper parameters (and their respective candidate values), it performs an exhaustive candidate hyperparameter combination search, returning to us the best hyper parameter combination for the `testing` set provided. It does this by iterating through all combinations of hyper parameters and evaluating those hyper parameters by k-fold cross validation. Each run giving an aggregated (mean) accuracy score of the model trained under the regulation of those hyperparameters, later used for comparing different hyperparameter combination effectiveness.

We in particular have to define the strategy used to evaluate the performance of each $\alpha$ tuned model... the list of strategies available in `GridSearchCV` are [listed here](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values). Since we are dealing with basic classification we opt for the `accuracy` scoring method, that is, given your $i$th predicted value $\hat y_i$ with the associated true label $y_i$, and keeping in mind the sample count $n_{samples}$;

$$accuracy(y,\hat y) = \frac{\sum_{i=0}^{n_{samples}-1}1(\hat y_i = y_i)}{n_{samples}}$$

TODO: We can go a step further and perform what is known as a nested cross-validation.

- [Chapter in `scikit-learn` that covers an exemplar nested cross-validation run](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html)
- [Video that helps describe the intuition behind nested cross validation](https://www.youtube.com/watch?v=az60jS7MQhU)


## Once we find this best $\alpha$

We build a final model, we call this one `best_performing_post_pruned_classifier` and use it as our strongest candidate that is using the "cost complexity pruning" technique. 

Some resources for further reading;

- [Chapter from `scitkit-learn` about hyper parameter tuning](https://scikit-learn.org/stable/modules/grid_search.html)
- [More practical to the point medium article on hyper parameter tuning](https://medium.com/chinmaygaikwad/hyperparameter-tuning-for-tree-models-f99a66446742)

_Note: in general, the more folds (`cv` param in `GridSearchCV`), the more representative the model accuracy would be of the real accuracy. The tradeoff here is patience, more accuracy requires more `cv` which in turn requires more time... I arbitrarilly chose 10 fold._

In [144]:
from sklearn.model_selection import GridSearchCV

print("Cost complexity pruning alpha candidates: ", len(path.ccp_alphas))
params = {'ccp_alpha': path.ccp_alphas}

base_classifier = tree.DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best")
search = GridSearchCV(estimator=base_classifier, n_jobs=-1, param_grid=params, cv=10, verbose=True, scoring='accuracy')
search.fit(tr_features, tr_labels)

print('Best "cost complexity alpha":', search.best_params_['ccp_alpha'])

best_performing_post_pruned_classifier = tree.DecisionTreeClassifier(random_state=0, criterion="entropy", splitter="best", ccp_alpha=GS.best_params_['ccp_alpha'])
best_performing_post_pruned_classifier.fit(tr_features, tr_labels)

print("Total number of trimmed nodes: ", unpruned_classifier.tree_.node_count - best_performing_post_pruned_classifier.tree_.node_count)

Cost complexity pruning alpha candidates:  232
Fitting 10 folds for each of 232 candidates, totalling 2320 fits
Best "cost complexity alpha": 0.0
Total number of trimmed nodes:  656


# (Task 4) Evaluating these Models
So now that we have iterated on the `post_pruned_classifier` and ended up with `best_performing_post_pruned_classifier`. It's time to formally evaluate these three models... the one mentioned and the basic `stump_classifier` and `unpruned_classifier` models. The absolute performance of these models are to be considered and you can quite easily fit each model on the `training` data provided and go ahead and predict the labels on these unseen `testing` features, finding the [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of each model and comparing this way (then there is the matter of evaluating precision, and recall etc... etc...).

In [145]:
from sklearn.model_selection import cross_validate

te_features, te_labels = testing.iloc[:, :-1], testing.iloc[:, -1]

predicted_labels = best_performing_post_pruned_classifier.predict(te_features)
score = accuracy_score(te_labels, predicted_labels)
print(f"Best performing post pruned classifier accuracy: %{score*100}")

predicted_labels = unpruned_classifier.predict(te_features)
score = accuracy_score(te_labels, predicted_labels)
print(f"Unpruned classifier classifier accuracy: %{score*100}")

predicted_labels = stump_classifier.predict(te_features)
score = accuracy_score(te_labels, predicted_labels)
print(f"Stump classifier classifier accuracy: %{score*100}")

Best performing post pruned classifier accuracy: %94.42267108833283
Unpruned classifier classifier accuracy: %96.47271630991861
Stump classifier classifier accuracy: %89.2372625866747


## Evaluating the statistical significance of pairwise model performance differences 

We should evaluate the difference between each model... as by chance a model could be performing well relative to another. How do we rule this chance out? We look at two models we're comparing, say $M_1$ and $M_2$ and set the null hypothesis to; $M_1$ are the same $M_2$, and using a [t-test](https://en.wikipedia.org/wiki/Student%27s_t-test), if we can reject the null hypothesis (ie, p-value < 0.05), then we can conclude the difference between $M_1$ and $M_2$ is **statistically significant**.

- [Quick video on the inards of the t-test for independent samples for those unfamiliar](https://www.youtube.com/watch?v=c9ombGmaEy8)

We need a series of like of like scoring methods, in our case we can keep using the standard `accuracy` scoring. We can generate these scores via something like a 10-fold cross validation, where we add the `accuracy` score of each iteration onto the series of scores $S_{M_1}$ and $S_{M_2}$ and utilise [`ttest_ind`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy-stats-ttest-ind) found within `scipy` in order calculate the T-test for the means of two independent samples of scores.

In [146]:
from sklearn.model_selection import cross_validate
from scipy import stats 

scores = cross_validate(best_performing_post_pruned_classifier, tr_features, tr_labels, scoring='accuracy', cv=10)
tuned_scores = scores['test_score']

scores = cross_validate(stump_classifier, tr_features, tr_labels, scoring='accuracy', cv=10)
stump_scores = scores['test_score']

t_stat, p_value = stats.ttest_ind(tuned_scores, stump_scores)
print("T-statistic value: ", t_stat)  
print("P-Value: ", p_value)

T-statistic value:  17.48032899609763
P-Value:  9.71112125247143e-13


# (Task 5) A Pre-Pruned Tree
In contrast to our section in "A Post Pruned Tree", we will look into using some pre-pruning techniques that actively restrict the growth of the tree as it builds. The hyperparameters we will look at will arbitrarilly be `max_depth` and to highlight further how `GridSearchCV` works, also consider in conjunction a series of `min_samples_leaf`.

Candidates for the following will be 